In [7]:
import csv
import json
#import logging
import re
import sys
import time
from datetime import datetime, timezone
from pathlib import Path
from urllib.parse import quote
from typing import List

import requests
import yaml
from dateutil import parser
import jsonschema
from jsonschema import validate

In [16]:
config_file = r"/Users/srappel/Documents/github/GeoDiscovery-Utils/opendataharvest/config.yaml"

try:
    with open(config_file, "r", encoding='utf-8') as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    print(f"Config file {config_file} not found")
    sys.exit()

try:
    CONFIG = config.get("CONFIG")
    OUTPUTDIR = Path("/Users/srappel/Documents/github/GeoDiscovery-Utils/opendataharvest/output_md/")
    LOGDIR = Path(CONFIG.get("LOGDIR"))
    DEFAULTBBOX = Path(CONFIG.get("DEFAULTBBOX"))
    CATALOG_KEY = CONFIG.get("CATALOG", "TestSites")
    CATALOG = config.get(CATALOG_KEY, None)
    MAXRETRY = CONFIG.get("MAXRETRY", 5)
    SLEEPTIME = CONFIG.get("SLEEPTIME", 1)

    # Default Values
    default_config = config.get("DEFAULT", {})
    MEMBEROF = default_config.get("MEMBEROF", [])
    RESOURCECLASS = default_config.get("RESOURCECLASS", [])
    ACCESSRIGHTS = default_config.get("ACCESSRIGHTS")  # This is a single string value
    MDVERSION = default_config.get("MDVERSION")  # This is a single string value
    LANG = default_config.get("LANG", [])
    PROVIDER = default_config.get("PROVIDER")  # This is a single string value
    SUPPRESSED = default_config.get("SUPPRESSED")  # This is a boolean value
    RIGHTS = default_config.get("RIGHTS", [])
    RESOURCETYPE = default_config.get("RESOURCETYPE", [])
    FORMAT = default_config.get("FORMAT")
    DESCRIPTION = default_config.get("DESCRIPTION")

    ## Get the JSON schema:
    SCHEMA = CONFIG.get("SCHEMA")

except AttributeError as e:
    print(f"Unable to read all configuration values from {config_file}")
    print(e)
    sys.exit()

dt = str(datetime.now().strftime(r"%Y%m%d%H%M%S"))
logfile_name = f"_{dt}.log"
LOGFILE = LOGDIR / logfile_name

In [17]:
class Site:
    """
    A class to represent a Site.

    Attributes
    ----------
    site_name : str
        The name of the site.
    site_details : dict
        The details of the site.
    site_json : dict
        The JSON data of the site.
    site_skiplist : set
        The set of UUIDs to skip.
    site_applist : set
        The set of UUIDs for applications.

    Methods
    -------
    __getitem__(self, key):
        Gets the attribute of the object using the key.
    __setitem__(self, key, value):
        Sets the attribute of the object using the key and value.
    """

    def __init__(
        self,
        site_name: str,
        site_details: dict,
        site_json: dict,
        site_skiplist: list,
        site_applist: list,
    ):
        """
        Constructs all the necessary attributes for the Site object.

        Parameters
        ----------
            site_name : str
                The name of the site.
            site_details : dict
                The details of the site.
            site_json : dict
                The JSON data of the site.
            site_skiplist : list
                The list of UUIDs to skip.
            site_applist : list
                The list of UUIDs for applications.
        """
        self.site_name = site_name
        self.site_details = site_details
        self.site_json = site_json
        self.site_skiplist = set(site_skiplist)
        self.site_applist = set(site_applist)

    def __getitem__(self, key):
        """
        Gets the attribute of the object using the key.

        Parameters
        ----------
            key : str
                The key to the attribute.

        Returns
        -------
            The value of the attribute.
        """
        return getattr(self, key)

    def __setitem__(self, key, value):
        """
        Sets the attribute of the object using the key and value.

        Parameters
        ----------
            key : str
                The key to the attribute.
            value : str
                The value to set the attribute to.
        """
        setattr(self, key, value)

In [18]:
def get_site_data(site: str, details: dict) -> dict:
    """Fetch the site data with retries."""
    for i in range(MAXRETRY):
        try:
            response = requests.get(details["SiteURL"], timeout=3)
            response.raise_for_status()
            return response.json()
        except json.JSONDecodeError:
            print(f"The content from {site} is not a valid JSON document.")
            return None
        except (requests.HTTPError, requests.exceptions.Timeout) as e:
            print(
                f"Received bad response from {site}. Retrying after {SLEEPTIME} seconds..."
            )
            time.sleep(SLEEPTIME)
            if i == (MAXRETRY - 1):
                print(
                    f"Failed to connect to {site} after {MAXRETRY + 1} attempts."
                )
                print(str(e))
                return None


def get_uuid_list(details: dict, key: str) -> List[str]:
    """Extract UUIDs from details."""
    uuid_list = []
    if key in details:
        for item in details[key]:
            uuid_list.append(item["UUID"])
    return uuid_list


def harvest_sites() -> list:
    """Main function to harvest sites."""
    site_list = []
    for site, details in CATALOG.items():
        site_json = get_site_data(site, details)
        if site_json is None:
            continue
        site_skiplist = get_uuid_list(details, "SkipList")
        site_applist = get_uuid_list(details, "AppList")
        current_Site = Site(
            details["SiteName"], details, site_json, site_skiplist, site_applist
        )
        site_list.append(current_Site)
    return site_list

In [19]:
class AardvarkDataProcessor:
    @staticmethod
    def extract_data(dataset_dict):
        # Extract data from dataset_dict
        title = dataset_dict.get("title", "Untitled Dataset")
        identifier = dataset_dict["identifier"]
        description = re.sub("<[^<]+?>", "", dataset_dict.get("description", []))
        creator = (
            [dataset_dict["publisher"]["name"]] if "publisher" in dataset_dict else []
        )
        issued = dataset_dict.get("issued", "")
        modified = dataset_dict.get("modified", "")
        keyword = dataset_dict.get("keyword", [])
        spatial = dataset_dict.get("spatial", None)
        distribution = dataset_dict.get("distribution", None)
        publisher = dataset_dict.get("publisher", [])
        landingPage = dataset_dict.get("landingPage", "")

        return {
            "title": title,
            "identifier": identifier,
            "description": description,
            "creator": creator,
            "issued": issued,
            "modified": modified,
            "keyword": keyword,
            "spatial": spatial,
            "distribution": distribution,
            "publisher": publisher,
            "landingPage": landingPage,
        }

    @staticmethod
    def extract_id_sublayer(url):
        id_pattern = r"id=([a-zA-Z0-9]+)"
        sublayer_pattern = r"sublayer=(\d+)"

        id_match = re.search(id_pattern, url)
        sublayer_match = re.search(sublayer_pattern, url)

        id_value = id_match.group(1) if id_match else None
        sublayer_value = sublayer_match.group(1) if sublayer_match else None

        if id_value is None:
            print(f"No id was extracted from the url: {url}")

        return id_value, sublayer_value

    @staticmethod
    def process_dcat_spatial(spatial_string):
        def is_in_range(value, range_min, range_max):
            return range_min <= value <= range_max

        # Extract coordinates
        pattern = r"(-?\d+\.\d+)"
        matches = re.findall(pattern, spatial_string)

        if len(matches) != 4:
            raise ValueError("Non-conforming spatial bounding box")

        # Convert to floats and validate coordinates
        coordinates = [float(coord) for coord in matches]
        longitudes = coordinates[::2]
        latitudes = coordinates[1::2]

        if not all(is_in_range(lon, -180, 180) for lon in longitudes):
            raise ValueError("Longitude coordinates must be between -180 and 180")

        if not all(is_in_range(lat, -90, 90) for lat in latitudes):
            raise ValueError("Latitude coordinates must be between -90 and 90")

        # Ensure North is greater than South and East is greater than West
        coordinates[1], coordinates[3] = sorted(latitudes, reverse=True)
        coordinates[0], coordinates[2] = sorted(longitudes)

        # Convert to ENVELOPE format
        envelope = f"ENVELOPE({coordinates[0]},{coordinates[2]},{coordinates[1]},{coordinates[3]})"

        return envelope

    @staticmethod
    def defaultBbox(website):
        envelope = None
        if "DefaultBbox" in website.site_details:
            default_bbox = website.site_details["DefaultBbox"]
            with open(DEFAULTBBOX) as default_csv:
                bboxreader = csv.DictReader(default_csv)
                for row in bboxreader:
                    if row["name"] == default_bbox:
                        envelope = f"ENVELOPE({row['west']},{row['east']},{row['north']},{row['south']})"
        return envelope

    @staticmethod
    def getURL(distribution):
        url = distribution.get("accessURL", distribution.get("downloadURL", "invalid"))
        return quote(url, safe=":/?=")

    @staticmethod
    def process_distribution(distribution):
        url = AardvarkDataProcessor.getURL(distribution)
        if "format" not in distribution or url == "invalid":
            return None

        format_to_reference = {
            "ArcGIS GeoServices REST API": {
                "FeatureServer": "urn:x-esri:serviceType:ArcGIS#FeatureLayer",
                "ImageServer": "urn:x-esri:serviceType:ArcGIS#ImageMapLayer",
                "MapServer": "urn:x-esri:serviceType:ArcGIS#DynamicMapLayer",
            },
            "ZIP": "http://schema.org/downloadUrl",
        }

        format_references = format_to_reference.get(distribution["format"], {})
        if isinstance(format_references, dict):
            for key, value in format_references.items():
                if key in url:
                    return {value: url}
        else:
            return {format_references: url}

        return None

    @staticmethod
    def format_fetcher(dataset_dict):
        for distribution in dataset_dict["distribution"]:
            dct_format_s = FORMAT
            gbl_resourceType_sm = RESOURCETYPE
            gbl_resourceClass_sm = ["Datasets"]
            if distribution["title"] == "Shapefile":
                dct_format_s = "Shapefile"
            elif "aerial" in dataset_dict.get("title", "").lower() or any(
                keyword.lower() in ["aerial photograph", "aerial imagery"]
                for keyword in dataset_dict.get("keyword", [])
            ):
                gbl_resourceType_sm[0] = "Aerial photographs"
                dct_format_s = "Raster data"
                if "Imagery" not in gbl_resourceClass_sm:
                    gbl_resourceClass_sm.append("Imagery")

        return dct_format_s, gbl_resourceType_sm, gbl_resourceClass_sm

    @staticmethod
    def issue_date_parser(dataset_dict):
        dt_string = dataset_dict["issued"]
        try:
            parsed_date = parser.parse(dt_string)
            dct_issued_s = parsed_date.strftime(r'%Y-%m-%d')
        except Exception as e:
            print(f'Unable to parse the year from: "{dt_string}". Error: {e}')
            dct_issued_s = dt_string

        return dct_issued_s

    @staticmethod
    def load_schema():
        response = requests.get(SCHEMA, timeout=3)
        schema = json.loads(response.text)
        return schema

    @staticmethod
    def validate_json(json_data, schema):
        try:
            validate(instance=json_data, schema=schema)
        except jsonschema.exceptions.ValidationError as err:
            return False, err
        return True, None


In [20]:
class Aardvark:
    """
    A class to represent a single dataset as an OGM Aardvark record
    """

    def __init__(self, dataset_dict, website):
        process_id_result = self._process_id(dataset_dict, website)
        if not process_id_result: # Dataset is in the skiplist
            return
        self._initialize_default_field_values()
        extracted_dataset_dict = AardvarkDataProcessor.extract_data(dataset_dict)
        self._process_extracted_dataset_dict(extracted_dataset_dict, website)


    def _initialize_default_field_values(self):
        self.pcdm_memberOf_sm = MEMBEROF
        self.gbl_resourceClass_sm = RESOURCECLASS
        self.dct_accessRights_s = ACCESSRIGHTS
        self.gbl_mdVersion_s = MDVERSION
        self.dct_language_sm = LANG
        self.schema_provider_s = PROVIDER
        self.gbl_suppressed_b = SUPPRESSED
        self.dct_rights_sm = RIGHTS

    def _process_id(self, dataset_dict, website):
        uuid, sublayer = AardvarkDataProcessor.extract_id_sublayer(
            dataset_dict["identifier"]
        )
        self.id = f"{website.site_name}-{uuid}{sublayer if sublayer else ''}"
        self.uuid = uuid

        if not self.id:
            print("ID is required.")
            return None

        # Stop processing if in skiplist
        if self.uuid in website.site_skiplist:
            print(f"{self.uuid} is on the skiplist...\n")
            return None

        self.dct_identifier_sm = [dataset_dict["identifier"]]
        return True

    def _process_extracted_dataset_dict(self, dataset_dict, website):
        self.dct_spatial_sm = website.site_details["Spatial"]

        prefix = website.site_details["CreatedBy"]
        title = prefix + " - " + dataset_dict["title"]
        self.dct_title_s = title

        self.gbl_mdModified_dt = datetime.now(timezone.utc).strftime(
            "%Y-%m-%dT%H:%M:%SZ"
        )

        self.dct_description_sm = [
            re.sub("<[^<]+?>", "", dataset_dict.get("description", []))
        ]
        self.dct_description_sm.append(DESCRIPTION)

        self.dct_creator_sm = (
            [dataset_dict["publisher"]["name"]] if "publisher" in dataset_dict else []
        )

        # dct_issued_s
        self.dct_issued_s = AardvarkDataProcessor.issue_date_parser(dataset_dict)

        self._process_spatial(dataset_dict, website)

        # dcat_keyword_sm (string multiple!)
        self.dcat_keyword_sm = dataset_dict["keyword"]

        self._process_distributions(dataset_dict)

        self._process_temporal_coverage(dataset_dict)

        # License and Rights
        rights = self.dct_rights_sm
        if dataset_dict.get("license"):
            rights.append(re.sub("<[^<]+?>", "", dataset_dict.get("license")))
        self.dct_rights_sm = rights

        # Format dct_format_s
        def set_attributes_if_not_none(obj, attr_values):
            attr_names = ["dct_format_s", "gbl_resourceType_sm", "gbl_resourceClass_sm"]
            for attr_name, attr_value in zip(attr_names, attr_values):
                if attr_value is not None:
                    setattr(obj, attr_name, attr_value)

        attr_values = AardvarkDataProcessor.format_fetcher(dataset_dict)
        set_attributes_if_not_none(self, attr_values)

        # Replace gbl_resourceClass_sm for web applications/websites
        if self.uuid in website.site_applist:
            self.gbl_resourceClass_sm = ["Websites"]

    def _process_spatial(self, dataset_dict, website):
        if "spatial" not in dataset_dict:
            print(f"No spatial information found for: {self.id}")
            return

        try:
            processed_spatial = AardvarkDataProcessor.process_dcat_spatial(
                dataset_dict["spatial"]
            )
            self.locn_geometry = self.dcat_bbox = processed_spatial
        except ValueError as e:
            print(
                f"There was a problem interpreting the bbox information for: {self.id}\n"
                f"\t - at {dataset_dict['landingPage']}\n"
                f"\t Error: {e}\n"
            )
            default_bbox = AardvarkDataProcessor.defaultBbox(website)
            if default_bbox is not None:
                self.locn_geometry = self.dcat_bbox = default_bbox
                print("Using default envelope for the website.\n")
            else:
                print(f"No default bounding box set for {website}")

    def _process_distributions(self, dataset_dict):
        if "distribution" not in dataset_dict:
            return

        references = {"http://schema.org/url": dataset_dict["landingPage"]}
        for distribution in dataset_dict["distribution"]:
            reference = AardvarkDataProcessor.process_distribution(distribution)
            if reference is not None:
                references.update(reference)

        self.dct_references_s = json.dumps(references).replace(" ", "")

    def _process_temporal_coverage(self, dataset_dict):
        if "modified" in dataset_dict:
            try:
                index_date = parser.parse(dataset_dict["modified"])
                index_year = int(index_date.year)
            except ImportError:
                index_year = int(dataset_dict["modified"][:4])
            except Exception as e:
                print(f"An error occurred: {e}")

            self.gbl_indexYear_im = [index_year]
            self.dct_temporal_sm = [f"Modified {index_year}"]
        else:
            self.gbl_indexYear_im = []

        if "issued" in dataset_dict:
            try:
                index_date = parser.parse(dataset_dict["issued"])
                index_year = int(index_date.year)
            except ImportError:
                index_year = int(dataset_dict["issued"][:4])
            except Exception as e:
                print("Problem processing the issued date.")

            self.gbl_indexYear_im.append(index_year)
            if self.dct_temporal_sm:
                self.dct_temporal_sm[0] = f"Issued {index_year}"
            else:
                self.dct_temporal_sm = [f"Issued {index_year}"]

    def to_dict(self):
        """
        Serialize the object to a dictionary, excluding None or empty values.
        """
        # List all the attributes that you want to include in the JSON output.
        attributes = [
            "id",
            "dct_title_s",
            "dct_creator_sm",
            "dct_identifier_sm",
            "dct_rights_sm",
            "gbl_resourceClass_sm",
            "dct_accessRights_s",
            "gbl_mdModified_dt",
            "gbl_mdVersion_s",
            "dct_language_sm",
            "schema_provider_s",
            "gbl_suppressed_b",
            "dct_spatial_sm",
            "dct_description_sm",
            "dct_issued_s",
            "dcat_keyword_sm",
            "dct_references_s",
            "dct_format_s",
            "gbl_resourceType_sm",
            "locn_geometry",
            "dct_temporal_sm",
            "gbl_indexYear_im",
        ]
        # Build the dictionary with attribute names and their values if they are not None or empty.
        return {
            attr: getattr(self, attr)
            for attr in attributes
            if hasattr(self, attr) and getattr(self, attr)
        }

    def __str__(self):
        # Use the to_dict method to get the dictionary representation of the object.
        obj_dict = self.to_dict()
        # Format the dictionary into a string for printing.
        return "\n".join(f"{key}: {value}" for key, value in obj_dict.items())

    def toJSON(self):
        aardvark_dict = self.to_dict()  # Use the new to_dict method
        json_dump = json.dumps(aardvark_dict)
        schema = AardvarkDataProcessor.load_schema()
        is_valid, error = AardvarkDataProcessor.validate_json(aardvark_dict, schema)
        if is_valid:
            return json_dump
        else:
            print(f"Failed JSON Validation:\n{error}")
            return

    def is_valid(self):
        json_dump = self.toJSON()  # Call toJSON as a method
        if json_dump is None:
            return False, "JSON serialization failed."

        json_object = json.loads(
            json_dump
        )  # Parse the JSON string back into a dictionary
        schema = AardvarkDataProcessor.load_schema()
        is_valid, error = AardvarkDataProcessor.validate_json(json_object, schema)
        if is_valid:
            return True, None
        else:
            return False, error

In [21]:
# Main Function
def main():
    list_of_sites = harvest_sites()

    for website in list_of_sites:
        new_aardvark_objects = [
            Aardvark(dataset, website) for dataset in website.site_json["dataset"]
        ]
        for new_aardvark_object in new_aardvark_objects:
            if new_aardvark_object.uuid not in website.site_skiplist:
                newfile = f"{new_aardvark_object.id}.json"
                newfilePath = OUTPUTDIR / newfile
                with open(newfilePath, "w", encoding='utf-8') as f:
                    json_data = new_aardvark_object.toJSON()
                    if not json_data is None:
                        f.write(new_aardvark_object.toJSON())
                    else:
                        print(f"{str(newfilePath)} not written...")


if __name__ == "__main__":
    main()

84c7b8d95af04cdda6b0c2ae26590531 is on the skiplist...

bce9201dd312445b9b4567ee14d8032a is on the skiplist...

d7f707071cd24b83ab3b9adb8a7d10ce is on the skiplist...

7a1d3d055d4b4457845c721088c132f0 is on the skiplist...



KeyboardInterrupt: 

In [15]:
import os

# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

/Users/srappel/Documents/github/GeoDiscovery-Utils/opendataharvest
